# final assignment: portfolio and alpaca


Choose a collection of assets. Using simulation, find the weights that will lead to the minimum variance portfolio. Make a python function that will purchase the correct amount of assets on Alpaca.

In [57]:
#pip install alpaca-trade-api

  Obtaining dependency information for alpaca-trade-api from https://files.pythonhosted.org/packages/84/5e/4912930d8f64a4a8c425b2a277849cc339a22b2046d3ecd7cbe6743f0021/alpaca_trade_api-3.0.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.6/332.6 kB 5.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aioh

In [2]:
import yfinance as yf
import numpy as np

from scipy.optimize import minimize 
#not installed yet

import requests
import json
import alpaca_trade_api as tradeapi

# Choose a collection of assets

In [3]:
tickers = ['SLB',  'OXY', 'BP',  'WMB',  'BKR', 'APA', 'HES']

# Step 1 Get Optimal Weights


In [8]:
# 获取股票数据
data = yf.download(tickers, start='2000-01-01', end='2023-09-07')['Adj Close']

# 计算每只股票的日收益率
returns = data.pct_change().dropna()

# 计算协方差矩阵
cov_matrix = returns.cov()

# 定义目标函数，最小化方差
def objective(weights):
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    return portfolio_variance

# 设置约束条件，这里假设权重之和为1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# 设置初始权重猜测值
#一开始我是用的initial_weights = np.ones(len(tickers)) / len(tickers)
#但是后来发现权重差不多，我就改成随机的了，可以看看再讨论
initial_weights = np.random.rand(len(tickers))
initial_weights /= initial_weights.sum() 

# 最小化目标函数，使用不同的优化方法
result = minimize(objective, initial_weights, constraints=constraints, method='trust-constr')


# 最优权重
optimal_weights = result.x

print("Optimal weights：", optimal_weights)
print("minimum variance：", result.fun)


[*********************100%%**********************]  7 of 7 completed


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Optimal weights： [0.18515656 0.15149588 0.19303193 0.05207661 0.15848791 0.16238412
 0.09736699]
minimum variance： 0.00045489873425015527


# Step 2 Get Account Balance

In [9]:
#与羊驼关联，到时候改一下自己的账户，我的注册失败了现在是老师的
API_KEY = "PKFYF7F8G1FOJ0MOCXZ2"
SECRET_KEY = "PydaHZeM1OAfZwroinrmgyMMNDaKOFYtaKiSvAqo"
BASE_URL = "https://paper-api.alpaca.markets"
ACCOUNT_URL = BASE_URL + "/v2/account"
ORDERS_URL =  BASE_URL + "/v2/orders"

api_key = "PK049ACO15YA2ADAOQZZ"
secret_key = "hzRNijPYEC0VGpcJDuFSXF6qAyp6GaMXOAhDF6Eb"

API_KEY = api_key
SECRET_KEY = secret_key


In [10]:
r = requests.get(ACCOUNT_URL, headers = {'APCA-API-KEY-ID': API_KEY,
                                         'APCA-API-SECRET-KEY':  SECRET_KEY})
info = json.loads(r.content)
accountval = float(info["cash"])

# Step 3 Get number of Shares

In [42]:
a = 1.0/len(tickers)

[0.1429, 0.1429, 0.1429, 0.1429, 0.1429]

In [9]:
# 计算投资组合价值
#一开始没写这行的时候都是负的，我感觉是不是老师账户金额设置的问题，我先写上这个了到时候关联上自己的账户可以删

#set the account value
accountval = 3e5# 假设账户总价值为1,000,000美元

optimal_weights  = [1/len(tickers)]*len(tickers)

# 计算每个股票的股票数量
shares = []
for i, symbol in enumerate(tickers):
    weight = optimal_weights[i]
    price = np.random.randint(1,100)
    #price = data[symbol][-1]  # 获取最后一个交易日的股价
    qty = (weight * accountval) / price
    qty = qty//1  # 取整数部分
    shares.append(qty)

for i, symbol in enumerate(tickers):
    print(f"The optimal number of stock {symbol} : {shares[i]}")

The optimal number of stock SLB : 1863.0
The optimal number of stock OXY : 714.0
The optimal number of stock BP : 8571.0
The optimal number of stock WMB : 996.0
The optimal number of stock BKR : 451.0
The optimal number of stock APA : 996.0
The optimal number of stock HES : 2040.0


# Step 4 Purchase amounts

In [12]:
def create_order(symbol, qty, side, type_, time_in_force):
    data = {
            "symbol": symbol,
            "qty": qty,
            "side": side,
            "type": type_,
            "time_in_force": time_in_force
            }
    r = requests.post(ORDERS_URL, json=data, headers = {'APCA-API-KEY-ID': API_KEY, 
                                                        'APCA-API-SECRET-KEY':  SECRET_KEY})

    return json.loads(r.content)

for i, symbol in enumerate(tickers):
    qty = shares[i]

    create_order(symbol, qty, "buy", "market", "gtc")

# REBALANCE PORTFOLIO TO CHECK THAT THE WEIGHTS ARE ALIGNED EACH DAY

In [4]:
# 计算当前股票数量
current_shares = []
for i, symbol in enumerate(tickers):
    price = latest_prices[symbol]
    qty = (accountval / len(tickers)) / price
    qty = int(qty)  # 取整数部分
    current_shares.append(qty)

# 打印当前股票数量
for i, symbol in enumerate(tickers):
    print(f"The current number of stock {symbol} : {current_shares[i]}")

# 计算最优股票数量与当前股票数量之间的差异
difference = [optimal_shares[i] - current_shares[i] for i in range(len(tickers))]

# 打印差异
for i, symbol in enumerate(tickers):
    print(f"The difference of stock {symbol} : {difference[i]}")

NameError: name 'latest_prices' is not defined

In [59]:
# Execute Trades to accoujnt for the difference
#显示资金不足，我觉得还是老师账户的问题
api = tradeapi.REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')
account_info = api.get_account()
account_cash = float(account_info.cash)

#why purchase like that why don't 
#simply use create order function
#to balance it: if positive, buy; negative sell.

for i, symbol in enumerate(tickers):
    qty = difference[i]
    if qty > 0:
        # 需要买入股票
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side='buy',
            type='market',
            time_in_force='gtc'
        )
        print(f"买入 {qty} 股 {symbol}")
    elif qty < 0:
        # 需要卖出股票
        api.submit_order(
            symbol=symbol,
            qty=-qty,
            side='sell',
            type='market',
            time_in_force='gtc'
        )
        print(f"卖出 {qty} 股 {symbol}")


APIError: insufficient buying power